<a href="https://colab.research.google.com/github/cesarofuchi/python_projects/blob/master/TabelaSanepar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geração de tabela com rodizio de água por bairro em Curitiba


A seca pelo qual Curitiba e região metropolitana esta passando pode estar longe de acabar. A data do rodízio intermitente de água é bem noticiada, porém não é avisado regularmente no meu condominio. 
 
![picture](https://github.com/cesarofuchi/python_projects/blob/main/sanepar/img/banner_20porcento.jpg?raw=true)
*texto em itálico*

Assim, decidi fazer um script para retirar da tabela oficial da agência de notícias do Paraná http://www.aen.pr.gov.br/ as informações para criar um relatório das datas do rodizio. 

Para que isso funcione, é preciso atualizar o link da tabela que esta em formato pdf

Primeiramente devemos fazer o download da tabela para deixar aqui na maquina virtual. 

Vou usar a biblioteca requests para fazer uma requisição dos dados e depois salvar na máquina local.




In [4]:
import requests
#import textract
response = requests.get("http://www.aen.pr.gov.br/arquivos/0910rodizioatualizada.pdf")
#myfile = open("out.pdf", "w")

with open('document.pdf', 'wb') as fw:
    fw.write(response.content)


Para extrar e manipular os dados da tabela será utilizada a biblioteca PDF Plumber.

Lembrando: Nativamente, o Google Colab não possui essa biblioteca instalada, então nós precisamos instalar a mesma no nosso ambiente.

Para isso, vamos utilizar o gerenciador de pacotes pip, e com o comando install vamos instalar a biblioteca.

In [5]:
!pip install pdfplumber

     |████████████████████████████████| 5.6MB 6.2MB/s 
     |████████████████████████████████| 143kB 38.4MB/s 
     |████████████████████████████████| 13.7MB 262kB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.23-cp36-none-any.whl size=29176 sha256=a6e4d55a00babdefbdc328c0a2b0ef653512f7ff98fd15cb06d0dd8a2d24fb41
  Stored in directory: /root/.cache/pip/wheels/f6/4d/ae/abb7b92fbed5d09a835c259f26d2a3223ea929a0ad6d322097
Successfully built pdfplumber


In [6]:
#biblioteca para manipular arquivos
import pdfplumber

In [7]:
relatorio = pdfplumber.open("document.pdf")

In [8]:
# abrir a primeira pagina do relatorio

pagina = relatorio.pages[0]
texto = pagina.extract_text()

In [9]:
tabela=relatorio.pages[1].extract_table()

In [24]:
#iterar por todas as paginas e concatenar as tabelas
table=[]
for i in range(len(relatorio.pages)):  
  pagina=relatorio.pages[i]
  table=table+pagina.extract_table()

  #pagina=relatorio.pages[i]
  

In [25]:
len(table)

408

In [45]:
# o que tem em uma linha da tabela
#table[2][0]=='GCBR'
#procura por codigo
str_procura='GCBR'
coluna=0
#procura por bairro (não funciona pois o mesmo bairro tem diferentes rodizios)
#str_procura="Batel"

#procura por reservatório
str_procura="área do Recalque Alto do Reservatório Cajuru"
str_procura="área da Gravidade do Reservatório Batel"
coluna=5

resultado=[]
j=0
for i in range(len(table)):
  if table[i][coluna] is not None:
    if table[i][coluna].find(str_procura)>=0:
      # resultado encontrado, separar os dados de inicio e fim do rodizio
      resultado.append([table[i][2],table[i][4]])
      local=table[i][5]
      #print('inicio=',table[i][2],' fim=', table[i][4])
      #print(table[i])
#adicionando nomes para as colunas
resultado.insert(0,['Início','Fim'])

In [46]:
resultado
local

'Curitiba (área da Gravidade do Reservatório Batel): Bigorrilho, Camp.Siqueira, Semináro, \nBatel, Mercês, Centro.'

Agora que sabemos uma forma de separar os dados, vamos buscar a informação que estamos buscando: Rodízio da minha região.

Analisando o output de cima, conseguimos identificar as datas pelo grupo

Aqui, vamos fazer o split, como havíamos feito anteriormente, mas somente na linha [6].
Tarefa 1 : Encontrar o grupo
A informação de grupo esta na último caracter de uma linha que tenha data no começo. 

   



In [17]:
!pip install reportlab

     |████████████████████████████████| 2.6MB 2.7MB/s 


In [47]:
#colando imagem no relatório

response = requests.get("https://github.com/cesarofuchi/python_projects/blob/main/sanepar/img/banner_20porcento.jpg?raw=true")
#myfile = open("out.pdf", "w")

with open('image.jpg', 'wb') as fw:  
    fw.write(response.content)

In [48]:
from reportlab.pdfgen import canvas
from reportlab.platypus import Frame, Image
from reportlab.lib.units import cm
from reportlab.lib import colors
from reportlab.platypus import Paragraph, Frame, Table, Spacer, TableStyle
from reportlab.lib.styles import getSampleStyleSheet



#def GeneratePDF(lista):


lista = resultado

#GeneratePDF(lista)

#    try:
nome_pdf = "rodizio_sanepar" #input('Informe o nome do PDF: ')
c = canvas.Canvas('{}.pdf'.format(nome_pdf))
# padrão é formato A4 (21 cm)
# posição em CM, posição (0,0) canto inferior esquerdo
c.setTitle(nome_pdf)
c.setFont("Helvetica-Bold", 15)

# colocar tabela
rtable=Table(resultado)
# propriedades da tabela
table_size=16
table_leading=table_size+2
rtable.setStyle(TableStyle([
  ('LEADING', (0, 0), (-1, -1), table_leading),
  ('SIZE', (0, 0), (-1, -1), table_size),
  ('INNERGRID', (0, 0), (-1, -1), 0.25, colors.black),
  ('BOX', (0, 0), (-1, -1), 0.25, colors.black)
]))

story = [Paragraph("RODÍZIO DE ABASTECIMENTO", getSampleStyleSheet()['Heading1']),
         Spacer(1, 5),
         Paragraph(local, getSampleStyleSheet()['Heading2']),
         Spacer(1, 5),
         rtable]

f = Frame(3*cm, 17*cm, 15 * cm, 9 * cm)
f.addFromList(story,c)
c.drawImage("/content/image.jpg", 6*cm, 7*cm, width=12*cm, preserveAspectRatio=True)

c.save()
print('{}.pdf criado com sucesso!'.format(nome_pdf))


rodizio_sanepar.pdf criado com sucesso!
